### Sample Script Prediction Script

Verify that loading and prediciton of the model is working correcticly

Hardware Setup:

    Memory (RAM): 31,4 GiB
    Processor - AMD® Ryzen 7 1700 eight-core processor × 16
    Graphics - GeForce GTX 1080 Ti/PCIe/SSE2
    OS type - 64-bit


In [ ]:
import mxnet as mx
import chess.variant
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.insert(0,'../../../')
from DeepCrazyhouse.src.runtime.color_logger import enable_color_logging
from DeepCrazyhouse.src.domain.util import *
from DeepCrazyhouse.src.domain.variants.crazyhouse.input_representation import board_to_planes
from DeepCrazyhouse.src.domain.variants.output_representation import policy_to_moves
from DeepCrazyhouse.src.domain.agent.neural_net_api import NeuralNetAPI
from DeepCrazyhouse.src.domain.variants.constants import *
from DeepCrazyhouse.configs.main_config import main_config
from collections import deque
from glob import glob
from time import time, sleep
%matplotlib inline
enable_color_logging()

In [ ]:
context = 'cpu'  # 'gpu'

In [ ]:
sym_path = glob(main_config['model_architecture_dir'] + '*')[0]
sym_path

In [ ]:
params_path = glob(main_config['model_weights_dir'] + '*')[0]
params_path

In [ ]:
sym = mx.sym.load(sym_path)

In [ ]:
# https://github.com/apache/incubator-mxnet/issues/6951
save_dict = mx.nd.load(params_path)
arg_params = {}
aux_params = {}
for k, v in save_dict.items():
    tp, name = k.split(':', 1)
    if tp == 'arg':
        arg_params[name] = v
    if tp == 'aux':
        aux_params[name] = v

In [ ]:
import time
import os

batch_shape = (8, 34, 8, 8)

### MXNet Baseline Performance

In [ ]:
# Create sample input
input = mx.nd.zeros(batch_shape)

In [ ]:
# Execute with MXNet
os.environ['MXNET_USE_TENSORRT'] = '0'
if context == 'cpu':
    ctx = mx.cpu()
else:
    ctx = mx.gpu(0)
executor = sym.simple_bind(ctx=ctx, data=batch_shape, grad_req='null', force_rebind=True)
executor.copy_params_from(arg_params, aux_params)

# Warmup
print('Warming up MXNet')
for i in range(0, 10):
    y_gen = executor.forward(is_train=False, data=input)
    y_gen[0].wait_to_read()

# Timing
print('Starting MXNet timed run')
start = time.process_time()
it = 100
for i in range(0, it):
    y_gen = executor.forward(is_train=False, data=input)
    y_gen[0].wait_to_read()
end = time.time()
print(f"Inference time per batch: {((time.process_time() - start) / it) * 1000}ms")

In [ ]:
%%timeit
input = mx.nd.array(np.zeros(batch_shape))
y_gen = executor.forward(is_train=False, data=input)
y_gen[0].wait_to_read()

### MXNet with TensorRT Integration Performance

In [ ]:
# Execute with TensorRT
print('Building TensorRT engine')
os.environ['MXNET_USE_TENSORRT'] = '1'
arg_params.update(aux_params)
all_params = dict([(k, v.as_in_context(mx.gpu(0))) for k, v in arg_params.items()])
executor = mx.contrib.tensorrt.tensorrt_bind(sym, ctx=mx.gpu(0), all_params=all_params,
                                             data=batch_shape, grad_req='null', force_rebind=True)

In [ ]:
%%timeit
input = mx.nd.array(np.zeros(batch_shape))
y_gen = executor.forward(is_train=False, data=input)
y_gen[0].wait_to_read()

In [ ]:
mod = mx.sym.load(sym_path)

In [ ]:
net_api = NeuralNetAPI(ctx=context)

In [ ]:
import chess

In [ ]:
board = chess.variant.CrazyhouseBoard()
board

In [ ]:
batch_size = 8
x = board_to_planes(board, board_occ=0, normalize=True)
x_mat_single = np.expand_dims(x, axis=0)
x_mat_batch = np.stack([x]*batch_size)

In [ ]:
x.shape

In [ ]:
pred = executor.forward(is_train=False, data=mx.nd.array(x_mat_batch, ctx=ctx))

In [ ]:
value, policy = pred[0][0].asnumpy(), pred[1][0].asnumpy()

### Show the output prediction

In [ ]:
[value, policy] = net_api.predict_single(x)

In [ ]:
value

In [ ]:
LABELS[policy.argmax()]

In [ ]:
opts = 5
selected_moves, probs = policy_to_moves(board, policy)
selected_moves[:opts]

In [ ]:
plt.barh(range(opts)[::-1], probs[:opts])
ax = plt.gca()
ax.set_yticks(range(opts)[::-1])
ax.set_yticklabels(selected_moves)